In [3]:
import numpy as np
from nltk.tokenize import word_tokenize
from lstm_vae import create_lstm_vae, inference
import keras
import sys, time
from keras.callbacks import CSVLogger, Callback

from tensorflow import set_random_seed
set_random_seed(1234)
np.random.seed(1234)


def get_text_data(data_path, num_samples=1000):

    # vectorize the data
    input_texts = []
    input_characters = set(["\t"])

    with open(data_path, "r", encoding="utf-8") as f:
        #lines = f.read().lower().split('\n')
        lines = f.read().lower().split('\n')
        
    for line in lines[: min(num_samples, len(lines) - 1)]:

        #input_text, _= line.split('\t')
        input_text = line
        input_text = word_tokenize(input_text)
        input_text.append("<end>")

        input_texts.append(input_text)

        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)

    input_characters = sorted(list(input_characters))
    num_encoder_tokens = len(input_characters)
    max_encoder_seq_length = max([len(txt) for txt in input_texts]) + 1

    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())

    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    decoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

    for i, input_text in enumerate(input_texts):
        decoder_input_data[i, 0, input_token_index["\t"]] = 1.0

        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
            decoder_input_data[i, t + 1, input_token_index[char]] = 1.0

    return max_encoder_seq_length, num_encoder_tokens, input_characters, input_token_index, reverse_input_char_index, \
           encoder_input_data, decoder_input_data



config_file_name = 'config.txt'

with open(config_file_name, 'r') as f:
    lines = f.readlines()

params = {}

for line in lines:
    line = line.split('\n')[0]
    param_list = line.split(' ')
    param_name = param_list[0]
    param_value = param_list[1]
    params[param_name] = param_value

num_samples = int(params['num_samples'])
data_path = "data/" + params['dataset']
dataname = params['dataset'].split('.')[0]

batch_size = int(params['batch_size'])
latent_dim = int(params['latent_dim'])
intermediate_dim = int(params['intermediate_dim'])
epochs = int(params['epochs'])

train = int(params['train'])
save = int(params['save'])
load = int(params['load'])

timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=num_samples,
                                                                                      data_path=data_path)

print(x.shape, "Creating model...")

input_dim = x.shape[-1]
timesteps = x.shape[-2]

if load:
    print("Loading model ... ")

    #vae = keras.models.load_model("models/vae_{}.h5".format(dataname))
    enc = keras.models.load_model("models/encoder_{}.h5".format(dataname))
    gen = keras.models.load_model("models/generator_{}.h5".format(dataname))
    stepper = keras.models.load_model("models/stepper_{}.h5".format(dataname))

if train:
    print("Training model...")

    vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim)


    csv_logger = CSVLogger('training_vae.log', separator=',', append=False)
    vae.fit([x, x_decoder], x, epochs=epochs, verbose=1, callbacks=[csv_logger])

    if save:
        print("Saving model ... ")

        vae.save("models/vae_{}.h5".format(dataname))
        enc.save("models/encoder_{}.h5".format(dataname))
        gen.save("models/generator_{}.h5".format(dataname))
        stepper.save("models/stepper_{}.h5".format(dataname))

print("Fitted, predicting...")




Number of samples: 300
Number of unique input tokens: 1386
Max sequence length for inputs: 90
(300, 90, 1386) Creating model...
Training model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None, 1386)   0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 353)          2456880     input_6[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 191)          67614       lstm_3[0][0]                     
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 191)          67614    

300/300 [==============================] - 29s 97ms/step - loss: 0.9497 - xent_loss: 0.9492 - kl_loss: 5.1988e-04
Epoch 46/200
300/300 [==============================] - 29s 97ms/step - loss: 0.9439 - xent_loss: 0.9434 - kl_loss: 4.9634e-04
Epoch 47/200
300/300 [==============================] - 29s 97ms/step - loss: 0.9374 - xent_loss: 0.9369 - kl_loss: 4.9010e-04
Epoch 48/200
300/300 [==============================] - 29s 97ms/step - loss: 0.9306 - xent_loss: 0.9301 - kl_loss: 4.8455e-04
Epoch 49/200
300/300 [==============================] - 29s 97ms/step - loss: 0.9234 - xent_loss: 0.9229 - kl_loss: 5.1902e-04
Epoch 50/200
300/300 [==============================] - 29s 97ms/step - loss: 0.9167 - xent_loss: 0.9162 - kl_loss: 5.1326e-04
Epoch 51/200
300/300 [==============================] - 29s 96ms/step - loss: 0.9086 - xent_loss: 0.9082 - kl_loss: 4.3928e-04
Epoch 52/200
300/300 [==============================] - 29s 97ms/step - loss: 0.9024 - xent_loss: 0.9019 - kl_loss: 4.3715e-

300/300 [==============================] - 29s 97ms/step - loss: 0.4483 - xent_loss: 0.4480 - kl_loss: 3.0993e-04
Epoch 110/200
300/300 [==============================] - 29s 97ms/step - loss: 0.4420 - xent_loss: 0.4417 - kl_loss: 3.0158e-04
Epoch 111/200
300/300 [==============================] - 29s 97ms/step - loss: 0.4328 - xent_loss: 0.4325 - kl_loss: 2.8335e-04
Epoch 112/200
300/300 [==============================] - 29s 97ms/step - loss: 0.4246 - xent_loss: 0.4243 - kl_loss: 2.6914e-04
Epoch 113/200
300/300 [==============================] - 29s 97ms/step - loss: 0.4168 - xent_loss: 0.4165 - kl_loss: 2.6466e-04
Epoch 114/200
300/300 [==============================] - 29s 97ms/step - loss: 0.4104 - xent_loss: 0.4102 - kl_loss: 2.4455e-04
Epoch 115/200
300/300 [==============================] - 29s 97ms/step - loss: 0.4055 - xent_loss: 0.4052 - kl_loss: 2.4420e-04
Epoch 116/200
300/300 [==============================] - 29s 97ms/step - loss: 0.3971 - xent_loss: 0.3968 - kl_loss: 2

300/300 [==============================] - 29s 98ms/step - loss: 0.1499 - xent_loss: 0.1497 - kl_loss: 1.6892e-04
Epoch 174/200
300/300 [==============================] - 29s 97ms/step - loss: 0.1473 - xent_loss: 0.1471 - kl_loss: 1.7724e-04
Epoch 175/200
300/300 [==============================] - 29s 98ms/step - loss: 0.1451 - xent_loss: 0.1449 - kl_loss: 1.6867e-04
Epoch 176/200
300/300 [==============================] - 29s 98ms/step - loss: 0.1430 - xent_loss: 0.1428 - kl_loss: 1.6377e-04
Epoch 177/200
300/300 [==============================] - 29s 98ms/step - loss: 0.1417 - xent_loss: 0.1416 - kl_loss: 1.6604e-04
Epoch 178/200
300/300 [==============================] - 29s 98ms/step - loss: 0.1425 - xent_loss: 0.1423 - kl_loss: 1.7059e-04
Epoch 179/200
300/300 [==============================] - 29s 98ms/step - loss: 0.1394 - xent_loss: 0.1393 - kl_loss: 1.6883e-04
Epoch 180/200
300/300 [==============================] - 29s 98ms/step - loss: 0.1376 - xent_loss: 0.1374 - kl_loss: 1

/home/axeldelavergne/anaconda3/envs/first_env/lib/python3.5/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'dense_7/BiasAdd:0' shape=(?, 353) dtype=float32>, <tf.Tensor 'dense_7/BiasAdd:0' shape=(?, 353) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Fitted, predicting...


/home/axeldelavergne/anaconda3/envs/first_env/lib/python3.5/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'dense_7_1/BiasAdd:0' shape=(?, 353) dtype=float32>, <tf.Tensor 'dense_7_1/BiasAdd:0' shape=(?, 353) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/axeldelavergne/anaconda3/envs/first_env/lib/python3.5/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_9:0' shape=(?, 353) dtype=float32>, <tf.Tensor 'input_10:0' shape=(?, 353) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [5]:
def decode(s, start_char = "\t"):
    return inference.decode_sequence(s, gen, stepper, input_dim, char2id, id2char, timesteps_max, start_char = start_char)

def continue_seq(x_start):
    return inference.continue_sequence(x_start, gen, stepper, input_dim, char2id, id2char, timesteps_max)

for _ in range(5):

    id_from = np.random.randint(0, x.shape[0] - 1)
    id_to = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])
    m_to, std_to = enc.predict([[x[id_to]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    seq_to = np.random.normal(size=(latent_dim,))
    seq_to = m_to + std_to * seq_to

    print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_from], axis=1)]), "==")

    for v in np.linspace(0, 1, 7):
        print("%.2f\t" % (1 - v), decode(v * seq_to + (1 - v) * seq_from))

    print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_to], axis=1)]), "==")

for _ in range(20):
    id_sentence = np.random.randint(0, x.shape[0] - 1)

    n_words = np.sum(x[id_sentence])
    n_kept = np.random.randint(n_words//2, n_words-1)

    new_x = np.zeros((x[id_sentence].shape))
    new_x[:n_kept,:] = x[id_sentence,:n_kept,:]

    print("==  \t", " ".join([id2char[j] for j in np.argmax(new_x[:n_kept], axis=1)]), " ... \t\t ==")

    print("\t...\t", continue_seq(new_x))

    print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_sentence], axis=1)]), "==")


==  	 the man sprang from his chair and paced up and down the room in uncontrollable agitation <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
1.00	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
0.83	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
0.67	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
0.50	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
0.33	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
0.17	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
0.00	 i was still balancing the matter in my mind when a hansom cab drove up to briony lodge , and 
==  	 you may know the strict principles of her fa

	...	 right forefinger , i was seized with a keen desire to his friend , incisive reasoning i was 
==  	 to me , who knew his every mood and habit , his attitude and manner told their own story <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
==  	 yes , he continued , glancing out of the  ... 		 ==
	...	 case <end> 
==  	 yes , he continued , glancing out of the window <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
==  	 if you will be good enough to call to-morrow afternoon at three o'clock i should like to chat this  ... 		 ==
	...	 is very door <end> 
==  	 if you will be good enough to call to-morrow afternoon at three o'clock i should like to chat this little matter over with you <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	